 # 시계열 전처리 및 기본 모델링


# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as spst
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import *

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

## (2) 데이터 불러오기

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/SeoulBike_Simple.csv'
bike = pd.read_csv(path)
bike['Datetime'] = pd.to_datetime(bike['Datetime'] )
bike.rename(columns={'Rented Bike Count':'Count'}, inplace = True)
bike = bike.loc[bike['Datetime'].between('2018-06-11','2018-08-13', inclusive = 'left'),
                ['Datetime', 'Temperature', 'Humidity','Count']]
bike.reset_index(drop = True, inplace = True)

## (3) 데이터 둘러보기

In [ ]:
bike.describe(include = 'all').T

In [ ]:
# 마지막 14일의 그래프를 그려 봅시다.
size = 24 * 14
temp = bike.iloc[-size:]
plt.figure(figsize = (20,8))
plt.plot('Datetime', 'Count', data = temp)
plt.grid()
plt.show()

# 2.시계열 데이터 전처리

## (1) y 만들기

* 사전 관찰(look-ahead) : 미래의 어떤 사실을 안다는 뜻
* 사전 관찰 문제 :
    * 데이터를 통해 실제로 알아야 하는 시점보다 더 일찍 미래에 대한 사실을 알게 되는 문제.  
    * 사전관찰 문제가 있는 채로 모델링을 하게 되면, 놀라운 성능의 모델이 만들어짐. --> 그러나 실제로는 불가능한 상황.

* 그래서 y를 만들때 사전관찰문제가 발생되지 않도록 해야 함.
    * 예제는 2시간 후의 수요량을 예측하고자 합니다.
    * 이를 위해 y를 어떻게 만들어야 할까요?

In [ ]:
data = bike.copy()

In [ ]:
data['y'] = data['Count'].shift(-2)
display(data.head())
display(data.tail())

In [ ]:
# 제일 마지막 행은 삭제
data = data.iloc[:-2]

## (2) NaN 조치
* 시계열 데이터 NaN 조치 방법
    * 1) 이전값으로 채우기
    * 2) 보간법(앞,뒤 값으로 채우기)


* nan 확인

In [ ]:
data.isna().sum()

* 1) 이전값으로 채우기

In [ ]:
data.loc[data['Temperature'].isna()]

In [ ]:
# NaN 조회
data.iloc[1390:1400]

In [ ]:
# 채운 것처럼 조회(실제 값이 변경되지 않았음.)
data.fillna(method = 'ffill').iloc[1390:1400]

In [ ]:
# 실제 적용
data.fillna(method = 'ffill', inplace = True)
data.iloc[1390:1400]

### 실습 : y만들기, NaN 조치

In [ ]:
temp = bike.copy()

* 1) 5시간 후를 예측하기 위한 y를 만들어 봅시다.

* 2) NaN에 대해서 선형 보간법으로 채워봅시다.
    * .interpolate(method = 'linear')

## (3) 데이터 분할

### 1) x, y 나누기

In [ ]:
target = 'y'

x = data.drop([target, 'Datetime'], axis = 1) #제거할 때, date도 제거
y = data.loc[:, target]

### 2) train, val 분할
* train_test_split( x, y, test_size = , shuffle = False)
    * test_size : 소수 - 비율, 자연수 - 갯수
    * shuffle = False : 섞지 말고 데이터 끝에서 test_size 만큼 자르기

In [ ]:
# 마지막 3주간의 데이터를 검증셋으로 사용 : 24시간 * 21일
size = 24 * 21
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = size, shuffle = False)

# 3.Baseline 모델

## (1) 모델링

In [ ]:
# 아래 문법은 다음 챕터에서 배우니 여기서는 신경쓰지 맙시다.
model1 = sm.tsa.SARIMAX(y_train, order=(1,1,1)).fit()

In [ ]:
# AIC 평가
model1.aic

## (2) 검증

In [ ]:
pred1 = model1.forecast(size)

print('MAE  : ', mean_absolute_error(y_val, pred1))
print('MAPE : ', mean_absolute_percentage_error(y_val, pred1))
print('R2   : ', r2_score(y_val, pred1))

## (3) 결과 시각화

* pred를 시리즈로 바꾸고, 인덱스 맞추기

In [ ]:
pred1 = pd.Series(pred1, index = y_val.index)

* y_train, y_val(전체), pred 한꺼번에 시각화

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y_train, label = 'train')
plt.plot(y_val, label = 'val')
plt.plot(pred1, label = 'pred')

plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y_val, label = 'val')
plt.plot(pred1, label = 'pred')

plt.legend()
plt.grid()
plt.show()

# 4.평가 : 잔차분석

* 잔차에 대한 우리의 기대 : 화이트 노이즈
    * 자기상관성 없음 : ACF, PACF 그래프
    * 정규분포 : Shapiro-Wilk 검정
    * 평균과 분산이 일정(Stationary) : ADF 검정
* 만약 화이트 노이즈가 아니라면...
    * 더 찾아내야 할 패턴이 있다는 의미.

## (1) ACF, PACF

* 정상 데이터 및 자기상관관계가 없는 데이터라면
* ACF, PACF 그래프에서
    * 첫번째 lag 에서부터 하늘색 범위 안에 값이 위치해야 하고
    * 값의 등락에 대한 어떠한 패턴도 보이지 않아야 합니다.
* 그러나 실제 데이터에서는 그런 결과를 보기 쉽지 않습니다.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
residuals = model1.resid

In [ ]:
residuals

### 1) ACF(자기 상관함수)

In [ ]:
plot_acf(residuals, lags = 50)
plt.show()

In [ ]:
# 반복문으로 시차를 늘려가면서 데이터셋 만들기
res_df = pd.DataFrame({'residuals':residuals})

for i in range(1,21) :
    var = 'lag' + str(i)
    res_df[var] = res_df['residuals'].shift(i)

In [ ]:
res_df.head(20)

In [ ]:
# 각 시차간의 상관계수
res_df.corr()

### 2) PACF

In [ ]:
plot_pacf(residuals, lags = 50)
plt.show()

In [ ]:
# ACF, PACF를 한꺼번에 그려 봅시다.
lags = 50

fig,ax = plt.subplots(1,2, figsize = (15,5))
plot_acf(residuals, lags = lags, ax = ax[0])
plot_pacf(residuals, lags = lags, ax = ax[1])
plt.show()

## (2) 검정
검정 도구를 적용하여 의사결정하는 용도로만 사용합니다.

In [ ]:
import scipy.stats as spst
import statsmodels.api as sm

### 1) 정규성 검정 : Shapiro-Wilk 검정

* 귀무가설 : **정규 분포이다.** (p-value > 0.05)
* 대립가설 : 정규분포가 아니다.

In [ ]:
spst.shapiro(residuals)[1]

### 2) 정상성 검정 : ADF 검정

* 귀무가설 : 비정상(Non-Stationary) 데이터이다.
* 대립가설 : **정상(Stationary) 데이터**이다.(P-value <= 0.05)

In [ ]:
# ADF 테스트
sm.tsa.stattools.adfuller(residuals)[1]

# 5.함수로 만들기

* 자주 사용하게 될 코드를 함수로 생성합니다.
    * 결과 시각화
    * 잔차 분석

## (1) 결과 시각화

In [ ]:
def plot_model_result(y_train, y_val, pred) :
    pred = pd.Series(pred, index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

In [ ]:
plot_model_result(y_train, y_val, pred1)

## (2) 잔차분석

In [ ]:
def residual_diag(residuals, lags = 30) :
    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1], 5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1], 5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

In [ ]:
residual_diag(residuals, lags = 50)